In [1]:
"""Implementations of algorithms for continuous control."""
import functools
from jaxrl_m.typing import *

import jax
import jax.lax as lax
import jax.numpy as jnp
import numpy as np
import optax
from jaxrl_m.common import TrainState, target_update, nonpytree_field
from jaxrl_m.networks import DeterministicPolicy,Policy, Critic, ensemblize

import flax
import flax.linen as nn

from functools import partial



class SACAgent(flax.struct.PyTreeNode):
    rng: PRNGKey
    critic: (TrainState)
    actor: TrainState
    config: dict = nonpytree_field()
    
    
        
    

    @partial(jax.jit,static_argnames=('num_steps',))  
    
    
    def update_many_critics(agent,transitions: Batch,idxs:jnp.array,num_steps:int,R2):
    
        def update_one_critic(critic_params,critic_opt_state,idxs,
                            agent,transitions,num_steps):
            
            
            def one_update(agent,
                        critic_params,critic_opt_state,
                        batch: Batch):
                                  
                def critic_loss_fn(critic_params):
                    next_actions = agent.actor(batch['next_observations'])
                    concat_actions = jnp.concatenate([batch["actions"],next_actions])
                    concat_observations = jnp.concatenate([batch["observations"],batch["next_observations"]])
                    
                    concat_q = agent.critic(concat_observations, concat_actions,
                                            True,params=critic_params)
                    q,next_q = jnp.split(concat_q,2,axis=0) ## axis=1 for ensemble
                    
                    target_q = batch['rewards'] + agent.config['discount'] * batch['masks'] * next_q
                    target_q = jax.lax.stop_gradient(target_q)
                    
                    critic_loss = ((target_q-q)**2).mean()
                    
                    return critic_loss, {}
        

                grads,_ = jax.grad(critic_loss_fn, has_aux=True)(critic_params)
                updates, new_opt_state = agent.critic.tx.update(grads, critic_opt_state, critic_params)
                new_params = optax.apply_updates(critic_params, updates)

                
                return agent,new_params,new_opt_state
            
            
            get_batch = lambda transitions,idx : jax.tree_map(lambda x : x[idx],transitions)
                
            agent,critic_params,critic_opt_state = jax.lax.fori_loop(0, num_steps, 
                        lambda i, args: one_update(*args,get_batch(transitions,idxs[i])),
                        (agent,critic_params,critic_opt_state))
            
            return critic_params,critic_opt_state
        
        
        
        critic = agent.critic
        ###### Reset optimizer params ######
        opt_state = jax.vmap(critic.tx.init,in_axes=0)(agent.critic.params)
        ###### Reset critic params ######
        
        reset = lambda rng,params : critic.init(rng,
                                                agent.config["observations"], agent.config["actions"],False)["params"]
        no_reset = lambda rng,params: params
        f = lambda  mask,rng,params :lax.cond(mask,reset,no_reset,rng,params)
        mask = R2 < 0.0
        rngs = jax.random.split(agent.rng, 5)
        critic_params = jax.vmap(f,in_axes=(0,0,0))(mask,rngs,critic.params)
        ###################################
        tmp = partial(update_one_critic,agent=agent,transitions=transitions,num_steps=num_steps)
        critic_params,critic_opt_state = jax.vmap(tmp,in_axes=(0,0,0))(critic_params,opt_state,idxs)
        new_critic = critic.replace(params=critic_params,opt_state=critic_opt_state)
        agent = agent.replace(critic=new_critic)
        
    
        return agent,{}
       
    

        
    @jax.jit
    def update_actor(agent, batch: Batch,q_weights):
        new_rng, curr_key, next_key = jax.random.split(agent.rng, 3)

        def actor_loss_fn(actor_params,q_weights):
            
            actions = agent.actor(batch['observations'], params=actor_params)
            
            call_one_critic = lambda observations,actions,params : agent.critic(observations,actions,params=params)
            q = jax.vmap(call_one_critic,in_axes=(None,None,0))(batch['observations'], actions,agent.critic.params)
            q_weights = jax.nn.softmax(q_weights,axis=0)
            q = jnp.sum(q_weights*q,axis=0)
            
            actor_loss = (-q).mean()
            
            return actor_loss, {
                'actor_loss': actor_loss,
              
            }

        loss_fn = partial(actor_loss_fn,q_weights=q_weights)
        new_actor, actor_info = agent.actor.apply_loss_fn(loss_fn=loss_fn, has_aux=True)

        return agent.replace(rng=new_rng,actor=new_actor,), {**actor_info}
        


    @jax.jit
    def sample_actions(agent,observations: np.ndarray) -> jnp.ndarray:
        actions = agent.actor(observations)
       
        return actions
    
 

def create_learner(
                 seed: int,
                 observations: jnp.ndarray,
                 actions: jnp.ndarray,
                 actor_lr: float = 3e-4,
                 critic_lr: float = 3e-4,
                 hidden_dims: Sequence[int] = (256, 256),
                 discount: float = 0.99,
                 tau: float = 0.005,
            **kwargs):

        print('Extra kwargs:', kwargs)

        rng = jax.random.PRNGKey(seed)
        rng, actor_key, critic_key = jax.random.split(rng, 3)

        action_dim = actions.shape[-1]
        actor_def = DeterministicPolicy((64,64), action_dim=action_dim,final_fc_init_scale=1.0)

        actor_params = actor_def.init(actor_key, observations)['params']
        actor = TrainState.create(actor_def, actor_params, tx=optax.adam(learning_rate=actor_lr))
        
        critic_def = Critic(hidden_dims)
        critic_params = critic_def.init(critic_key, observations, actions,False)['params']
        tmp_critic = TrainState.create(critic_def, critic_params,optax.adam(learning_rate=critic_lr))
        
        critic_keys  = jax.random.split(critic_key, 5)
        critic_params = jax.vmap(critic_def.init,in_axes=(0,None,None))(critic_keys, observations, actions)['params']
        critic = tmp_critic.replace(params=critic_params) ## opt_state is updated in "update_many_critics"

        
        config = flax.core.FrozenDict(dict(
            discount=discount,
            target_update_rate=tau, 
            observations = observations,
            actions = actions,
        ))

        return SACAgent(rng, critic=critic, actor=actor, config=config)
        


/home/mahdi/Desktop/jaxrl_m/.venv/lib/python3.10/site-packages/jax/_src/api_util.py:229: SyntaxWarning: Jitted function has invalid argnames {'num_batches'} in static_argnames. Function does not take these args.This warning will be replaced by an error after 2022-08-20 at the earliest.
  warnings.warn(f"Jitted function has invalid argnames {invalid_argnames} "


In [2]:
from jaxrl_m.dataset import ReplayBuffer
from flax import struct
import chex



@struct.dataclass
class PolicyRollout:
    
    policy_params : chex.Array
    num_rollouts : chex.Array 
    policy_return : chex.Array
    observations : chex.Array
    disc_masks : chex.Array
    

def rollout_policy(agent,env,exploration_rng,
                   replay_buffer,actor_buffer,
                   warmup=False,num_rollouts=10,):
    
    
    actor_buffer.reset()
    obs,_ = env.reset()  
    n_steps,n_rollouts,episode_step,disc,mask = 0,0,0,1.,1.
    max_steps = num_rollouts*1000
    observations,disc_masks,rewards = np.zeros((max_steps,obs.shape[0])),np.zeros((max_steps,)),np.zeros((max_steps,))
    
    
    while n_rollouts < num_rollouts:
        
        if warmup:
            action = env.action_space.sample()
        else:
            exploration_rng, key = jax.random.split(exploration_rng)
            action = agent.select_action(obs)
        
        next_obs, reward, done, truncated, info = env.step(action)
        #reward = reward/400
        
        mask = float(not done)
    
        transition = dict(observations=obs,actions=action,
            rewards=reward,masks=mask,next_observations=next_obs,discounts=disc)
        
        
        replay_buffer.add_transition(transition)
        actor_buffer.add_transition(transition)
    
        observations[1000*n_rollouts+episode_step] = obs
        disc_masks[1000*n_rollouts+episode_step] = disc
        rewards[1000*n_rollouts+episode_step] = reward
        
        obs = next_obs
        disc *= (0.99*mask)
        episode_step += 1
        n_steps += 1
        
        if (done or truncated) :
            
            #exploration_metrics = {f'exploration/{k}': v for k, v in flatten(info).items()}
            obs,_= env.reset()
            n_rollouts += 1
            episode_step = 0
            disc,mask = 1.,1.

    policy_return = (disc_masks*rewards).sum()/num_rollouts
    policy_rollout = PolicyRollout(policy_params=agent.actor.params,
                                   policy_return=policy_return,
                                   observations=observations,
                                   disc_masks=disc_masks,
                                    num_rollouts=num_rollouts)
    
    return replay_buffer,actor_buffer,policy_rollout,policy_return,n_steps

In [3]:




def f(anc_agent,obs,actor):

    actions = anc_agent.actor(obs, params=actor)
    q1, q2 = anc_agent.critic(obs, actions,params=anc_agent.target_critic.params)
    #q = jnp.minimum(q1, q2)
    q = (q1+q2)/2
    
    return q
    

@jax.jit
def estimate_return(anc_agent,anc_return,acq_rollout:PolicyRollout,):
    
    acq_obs = acq_rollout.observations
    acq_masks = acq_rollout.disc_masks
    
    acq_actor = acq_rollout.policy_params
    acq_return = acq_rollout.policy_return
    
    anc_actor = anc_agent.actor.params
    
    acq_q = f(anc_agent,acq_obs,acq_actor)
    anc_q = f(anc_agent,acq_obs,anc_actor)
    
    adv = ((acq_q - anc_q)*acq_masks).sum()/10
    acq_return_pred = anc_return + adv
  
    
    return acq_return_pred,acq_return


def evaluate_critic(anc_agent,anc_return,policy_rollouts):

    y_pred,y= [],[]
    for policy_rollout in policy_rollouts:
        
        acq_return_pred,acq_return = estimate_return(anc_agent,anc_return,policy_rollout)
        y_pred.append(acq_return_pred),y.append(acq_return)
        
    y_pred,y = np.array(y_pred),np.array(y)
    a2 = jnp.clip(((y-y_pred)**2),a_min=1e-4).sum()
    b2=((y-y.mean())**2).sum()
    R2 = 1-(a2/b2)  
    
    return R2


In [4]:
import os
from functools import partial
import numpy as np
import jax
import tqdm
import gymnasium as gym

from jaxrl_m.common import CodeTimer
from jaxrl_m.wandb import setup_wandb, default_wandb_config, get_flag_dict
import wandb
from jaxrl_m.evaluation import supply_rng, evaluate, flatten, EpisodeMonitor
from jaxrl_m.dataset import ReplayBuffer
from collections import deque
from jaxrl_m.rollout import PolicyRollout,rollout_policy

env_name='Hopper-v4'
seed=np.random.choice(1000000)
eval_episodes=10
batch_size = 256
max_steps = int(1e6)
start_steps = 50000                   
log_interval = 5000
#eval_interval = 10000

wandb_config = default_wandb_config()
wandb_config.update({
    'project': 'd4rl_test',
    'group': 'sac_test',
    'name': 'sac_{env_name}',
})


env = EpisodeMonitor(gym.make(env_name))
eval_env = EpisodeMonitor(gym.make(env_name))
setup_wandb({"bonjour":1})

example_transition = dict(
    observations=env.observation_space.sample(),
    actions=env.action_space.sample(),
    rewards=0.0,
    masks=1.0,
    next_observations=env.observation_space.sample(),
    discounts=1.0,
)

replay_buffer = ReplayBuffer.create(example_transition, size=int(1e6))
actor_buffer = ReplayBuffer.create(example_transition, size=int(5e3))

agent = create_learner(seed,
                example_transition['observations'][None],
                example_transition['actions'][None],
                max_steps=max_steps,
                #**FLAGS.config
                )

exploration_metrics = dict()
obs,info = env.reset()    
exploration_rng = jax.random.PRNGKey(0)
i = 0
num_grad_updates = 0
unlogged_steps = 0
policy_rollouts = deque([], maxlen=10)
R2 = jnp.ones(5)
with tqdm.tqdm(total=max_steps) as pbar:
    
    while i < max_steps:


        #with CodeTimer('rollout'):
            
            replay_buffer,actor_buffer,policy_rollout,policy_return,undisc_policy_return,num_steps = rollout_policy(agent,env,exploration_rng,
                    replay_buffer,actor_buffer,
                    warmup=(i < start_steps))
            policy_rollouts.append(policy_rollout)
            unlogged_steps += num_steps
            i+=num_steps
            pbar.update(num_steps)
            
            
        
            if replay_buffer.size > start_steps:
        
        
            #with CodeTimer('update_critic'):
                key = jax.random.PRNGKey(0)
                transitions = replay_buffer.get_all()
                tmp = partial(jax.random.choice,a=replay_buffer.size, shape=(10000,256), replace=True)
                idxs = jax.vmap(tmp)(jax.random.split(key, 5))
                agent, critic_update_info = agent.update_many_critics(transitions,idxs,10000,R2)
            
    #        with CodeTimer('evaluate_critic'):        
                critic_params = agent.critic.params
                anc_return = policy_rollouts[-1].policy_return
                flattened_rollouts = flatten_rollouts(policy_rollouts)
                R2,bias = evaluate_critics(agent,critic_params,anc_return,flattened_rollouts)

            #with CodeTimer('update_actor'):
                actor_batch = actor_buffer.get_all()      
                agent, actor_update_info = agent.update_actor(actor_batch,R2.reshape(-1,1))   
                num_grad_updates += 1 
                
            
                
                if unlogged_steps > log_interval:
                    
                    #with CodeTimer('eval'):
                            
                        
                        update_info = {**critic_update_info, **actor_update_info,
                                'R2_validation': jnp.max(R2),'bias': jnp.max(bias),'num_grad_updates': num_grad_updates
                                }
                        exploration_metrics = {f'exploration/disc_return': policy_return}
                        
                        train_metrics = {f'training/{k}': v for k, v in update_info.items()}
                        
                        # eval_info = evaluate(agent.actor, eval_env, num_episodes=eval_episodes)
                        # eval_metrics = {f'evaluation/{k}': v for k, v in eval_info.items()}
                        
                    #with CodeTimer('logging'):
                        
                        wandb.log(train_metrics, step=int(i),commit=False)
                        wandb.log({"undisc_return":undisc_policy_return},step=int(i))
                        wandb.log(exploration_metrics, step=int(i),commit=True)
                        #wandb.log(eval_metrics, step=int(i),commit=True)
                        unlogged_steps = 0
                    

2023-12-15 01:12:56.377628: W external/xla/xla/service/gpu/nvptx_compiler.cc:679] The NVIDIA driver's CUDA version is 12.2 which is older than the ptxas CUDA version (12.3.103). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mahdikallel. Use `wandb login --relogin` to force relogin


Extra kwargs: {'max_steps': 1000000}


  1%|          | 5356/1000000 [00:04<16:00, 1035.45it/s] 

evaluating


  1%|          | 11222/1000000 [00:28<56:46, 290.27it/s]

...

evaluating


 99%|█████████▉| 990509/1000000 [12:37<00:06, 1487.04it/s]

evaluating


1000509it [12:47, 1303.28it/s]                            

evaluating


In [5]:
#opt_state = tx.init(params)

#dir(agent.critic)
new_opt_state = agent.critic.tx.init(agent.critic.params)
new_critic = agent.critic.replace(opt_state=new_opt_state)
